### 1. 解释了如何在Lambda表达式中捕获异常
### 2. 使用类似Maybe的装饰器处理Lambda表达式中的错误
### 3. 习题课 - - - 看过一个依赖于我们学到的许多技术的交互式计算器

## 1.

### Exceptions
- 在Python中，每当发生错误，异常对象被“抛出(raise)”时,需要处理异常. 这导致代码中止，直到在try：... except：...语句中捕获异常

In [1]:
def add_str(s):
    
    try:
        return sum([int(i) for i in s.split('+')])
    except AttributeError:
        return None
    
print(add_str('2+3'))
print(add_str(2+3))

5
None


### 来自 "语句" 的诅咒
- 但是try是一个语句，因此你不能在Lambda表达式中使用它！
- 与大多数其他语句不同，Python不提供替代try语句的表达式。

In [2]:
l_add_str = lambda s: sum([int(i) for i in s.split("+")])

print(l_add_str('2+5'))
# print(l_add_str(2+5))  # AttributeError: 'int' object has no attribute 'split'

7


## 2.

### 一个类似 " Maybe "  的装饰器
- 直接使用" Maybe monad " 并不是纯正的python风格 (Pythonic). 我们可以使用装饰器做类似的事情。

In [3]:
def maybe(fnc):
    
    def inner (*args):
        for arg in args:
            if isinstance(arg, Exception):
                return arg
        try:
            return fnc(*args)
        except Exception as e:
            return e
        
    return inner

safe_add_str = maybe(lambda s: sum([int(i) for i in s.split("+")]))
print(safe_add_str(1+2))

'int' object has no attribute 'split'


## 3.

### 习题课 - - - Our functional calculator... so far
This is the calculator that we implemented in the first section. But it suffers from a few drawbacks:
- No input validation
- No looping

In [4]:
OPERATORS = '+', '-', '*', '/'

def f_get_number():
    return int(input('Enter an integer:'))


def f_get_oparetor():
    return input("Enter an operator('+', '-', '*', '/')")


def f_calculate(number1, operator, number2):
    return number1+number2 if operator == '+' \
        else number1-number2 if operator == '-' \
        else number1*number2 if operator == '*'\
        else number1/number2 if operator == '/'\
        else None
    

def f_mian():
    return f_calculate(
    f_get_number(),
    f_get_oparetor(),
    f_get_number(),
    )
 
print('The result is : ', f_mian())

Enter an integer:1
Enter an operator('+', '-', '*', '/')+
Enter an integer:1
The result is :  2


### Lets get to work!
Our toolkit contains:
- Lambda expressions
- Decorators
- Higher-order functions

In [ ]:
OPERATORS = '+', '-', '*', '/'


def maybe(fnc):
    def inner(*args):
        for arg in args:
            if isinstance(arg, Exception):
                return arg
        try:
            return fnc(*args)
        except Exception as e:
            return e

    return inner


def repeat(fnc, until):
    def inner(*args):

        while True:
            result = fnc(*args)
            if until(result):
                return result

    return inner


is_int = lambda i: isinstance(i, int)
get_number = lambda: int(input('Enter an integer:'))
safe_get_number = repeat(maybe(get_number), until=is_int)

is_operator = lambda o: o in OPERATORS
get_operator = lambda: input("Enter an operator('+', '-', '*', '/')")
safe_get_operator = repeat(get_operator, until=is_operator)

calculate = lambda number1, operator, number2: \
    number1 + number2 if operator == '+' \
        else number1 - number2 if operator == '-' \
        else number1 * number2 if operator == '*' \
        else number1 / number2 if operator == '/' \
        else None

main = lambda: calculate(
    safe_get_number(),
    safe_get_operator(),
    safe_get_number(),
)

forever = lambda retval: False
main_loop = repeat(lambda: print(main()), until=forever)

main_loop()

Enter an integer:-
Enter an integer:1
Enter an operator('+', '-', '*', '/')+-*/
Enter an operator('+', '-', '*', '/')+
Enter an integer:1
2
